# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_data[["Lat", "Lng"]].astype(float)
humidity = city_data["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = city_data.loc[(city_data["Humidity"]>=70) & (city_data["Wind Speed"]>=10) & \
                                 (city_data["Cloudiness"] <= 20)].dropna()

narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,40,hermanus,9,ZA,1558378761,74,-34.42,19.24,55.00,23.78
54,54,tiarei,20,PF,1558378764,70,-17.53,-149.33,84.20,10.29
55,55,khatanga,10,RU,1558378764,92,71.98,102.47,9.96,10.94
66,66,dikson,12,RU,1558378766,85,73.51,80.55,20.22,12.15
102,102,ketchikan,1,US,1558378684,76,55.34,-131.65,55.40,17.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,40,hermanus,9,ZA,1558378761,74,-34.42,19.24,55.00,23.78,
1,54,tiarei,20,PF,1558378764,70,-17.53,-149.33,84.20,10.29,
2,55,khatanga,10,RU,1558378764,92,71.98,102.47,9.96,10.94,
3,66,dikson,12,RU,1558378766,85,73.51,80.55,20.22,12.15,
4,102,ketchikan,1,US,1558378684,76,55.34,-131.65,55.40,17.22,
5,103,kapaa,20,US,1558378773,74,22.08,-159.32,80.60,10.29,
6,117,east london,20,ZA,1558378775,72,-33.02,27.91,60.80,27.51,
7,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,
8,129,paita,0,PE,1558378778,290,-5.09,-81.11,62.60,17.22,
9,134,melfi,20,IT,1558378779,87,41.00,15.66,59.00,10.29,


In [7]:
# geocoordinates
target_search = "Hotel"
target_radius = 5000
target_type = "Hotels"

params={
    "radius":target_radius,
    "types":target_type,
    "keyword":target_search,
    "key":g_key
}

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))